In [1]:
import tensorflow as tf
import numpy as np

C:\Users\o00408152\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
global gama
gama = 0
states = 10
actions = 2
world = np.zeros(10)

sess = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, states], name='input_states')
y_ = tf.placeholder(tf.float32, shape=[None, actions], name='correct_answer')
W = tf.Variable(tf.zeros([10,2]), name='weights')
b = tf.Variable(tf.zeros([2]), name='bias')

# predicted action
y = tf.nn.softmax(tf.matmul(x,W) + b)

cross_antropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_antropy)

init = tf.global_variables_initializer()
sess.run(init)



In [3]:
def get_next_state(state, gama):
    # act greedy with random
    coin = np.random.randint(500 + gama)
    if coin > 400:
        a = get_best_action_from_nn(state)
    else:
        a = np.random.randint(actions)
    s = get_next_valid_state(state, a)

    return s, a

In [4]:
def get_next_valid_state(state, action):
    if action == 0:
        if state == 0:
            return state
        return state-1
    if state == 9:
        return state
    return state+1

In [5]:
def get_current_reward(state):
    if state == 0:
        return 0
    if state == 9:
        return 1
    return 0.5

In [6]:
def get_state_as_array(state):
    s = np.zeros(10, dtype=np.float32)
    s[state] = 1.0
    return s

In [7]:
def get_reward_from_nn(state, a):
    s = get_state_as_array(state)
    b = sess.run(y, feed_dict={x: [s]})
    return b[0,a]

In [8]:
def get_best_action_from_nn(state):
    s = get_state_as_array(state)
    a = sess.run(tf.arg_max(y,1), feed_dict={x: [s]})
    return a[0]    

In [9]:
def get_max_reward_from_nn(state):
    s = get_state_as_array(state)
    m = sess.run(tf.reduce_max(y,1), feed_dict={x: [s]})
    return m[0]

In [10]:
def get_all_rewards(state):
    s = get_state_as_array(state)
    b = sess.run(y, feed_dict={x: [s]})
    return b[0].tolist()    

In [11]:
def train(state, rewards):
    s = get_state_as_array(state)
    sess.run(train_step, feed_dict={x:[s], y_:[rewards]})

In [12]:
state = np.random.randint(1, states-1)

In [13]:
for i in range(100):
    r= []
    nn = []
    
    # update value function
    for action in range(actions):
        ns = get_next_valid_state(state, action)
        nn.append(ns)
        r.append(get_max_reward_from_nn(ns))
    pr = get_all_rewards(state)
    
    # update NN
    if not r == pr:
        train(state, r)
    
    # act greedy
    
    ls = state
    state, la = get_next_state(state, gama)
    
    # get current state value
    r = get_current_reward(state)
    
    # update privious state and action
    if r != 0.5:
        rewards = get_all_rewards(ls)
        rewards[la] = r
        train(ls, rewards)
            
    gama+=1


Instructions for updating:
Use `argmax` instead


In [16]:
yy = []
for i in range(10):
    yy.append(get_all_rewards(i))
yy
    

[[0.44866493344306946, 0.5513350367546082],
 [0.44866493344306946, 0.5513350367546082],
 [0.44866493344306946, 0.5513350367546082],
 [0.44866493344306946, 0.5513350367546082],
 [0.44866493344306946, 0.5513350367546082],
 [0.45134997367858887, 0.5486499667167664],
 [0.4525884985923767, 0.5474115014076233],
 [0.4388071596622467, 0.5611928701400757],
 [0.35374459624290466, 0.646255373954773],
 [0.4994153678417206, 0.500584602355957]]